In [2]:
import io
import re
import os
import sys
from os import listdir
from os.path import isfile, join,isdir

os.environ["PYSPARK_PYTHON"] = "/home/user/anaconda3/bin"
os.environ['PYSPARK_DRIVER_PYTHON']="/home/user/anaconda3/bin"
os.environ["SPARK_HOME"] = "/mnt/e/Data/Tools/apache/spark-3.5.3-bin-hadoop3/"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
os.environ["HADOOP_HOME"]= "/mnt/e/Data/Tools/apache/hadoop-3.3.6"
sys.path.insert(0, os.environ["SPARK_HOME"])
sys.path.insert(0, os.environ["SPARK_HOME"] +"/bin")
sys.path.insert(0, os.environ["HADOOP_HOME"] +"/bin")
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.9.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")


In [3]:
! python --version

Python 3.12.4


In [4]:
import findspark
findspark.init()
findspark.find()

'/mnt/e/Data/Tools/apache/spark-3.5.3-bin-hadoop3/'

In [33]:
import pyspark
#from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import regexp_extract, col, round
from pyspark.sql import SparkSession
import random


In [6]:
os.environ["JAVA_HOME"]="/u01/jdk1.8.0_351/"
os.environ["SPARK_LOCAL_IP"]="127.0.0.1"
os.environ["HADOOP_OPTS"] = "$HADOOP_OPTS -Djava.library.path=$HADOOP_HOME/lib/native"
sys.path.insert(0, os.environ["JAVA_HOME"] +"/bin")

In [7]:
spark = SparkSession.builder\
.master("local[5]") \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/14 23:55:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/14 23:55:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/14 23:55:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
usersdf = spark.read.options(delimiter=",", sep = ",", header=True, inferSchema=True,multiline=True, quote="\"", escape="\"").format("csv").load("/mnt/e/tmp/DZ1/users.csv")
purchasesdf = spark.read.options(delimiter=",", sep = ",", header=True, inferSchema=True,multiline=True, quote="\"", escape="\"").format("csv").load("/mnt/e/tmp/DZ1/purchases.csv")
productsdf = spark.read.options(delimiter=",", sep = ",", header=True, inferSchema=True,multiline=True, quote="\"", escape="\"").format("csv").load("/mnt/e/tmp/DZ1/products.csv")


In [9]:
usersdf.summary().show()

24/11/14 23:55:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-------+-----------------+-------------------+
|summary|           user_id|   name|              age|              email|
+-------+------------------+-------+-----------------+-------------------+
|  count|               100|     98|               98|                 99|
|   mean|              50.5|   NULL|35.13265306122449|               NULL|
| stddev|29.011491975882016|   NULL|9.168117947710689|               NULL|
|    min|                 1| User_1|               18|user100@example.com|
|    25%|                25|   NULL|               27|               NULL|
|    50%|                50|   NULL|               36|               NULL|
|    75%|                75|   NULL|               43|               NULL|
|    max|               100|User_99|               49|  user9@example.com|
+-------+------------------+-------+-----------------+-------------------+



In [10]:
usersdf.describe().show()

+-------+------------------+-------+-----------------+-------------------+
|summary|           user_id|   name|              age|              email|
+-------+------------------+-------+-----------------+-------------------+
|  count|               100|     98|               98|                 99|
|   mean|              50.5|   NULL|35.13265306122449|               NULL|
| stddev|29.011491975882016|   NULL|9.168117947710689|               NULL|
|    min|                 1| User_1|               18|user100@example.com|
|    max|               100|User_99|               49|  user9@example.com|
+-------+------------------+-------+-----------------+-------------------+



In [11]:
usersdf_clean=usersdf.dropna()
purchasesdf_clean=purchasesdf.dropna()
productsdf_clean=productsdf.dropna()

In [12]:
usersdf_clean.describe().show()

+-------+------------------+-------+-----------------+-------------------+
|summary|           user_id|   name|              age|              email|
+-------+------------------+-------+-----------------+-------------------+
|  count|                95|     95|               95|                 95|
|   mean|              49.6|   NULL|34.96842105263158|               NULL|
| stddev|28.691462144686874|   NULL|9.165676759472076|               NULL|
|    min|                 1| User_1|               18|user100@example.com|
|    max|               100|User_98|               49|  user9@example.com|
+-------+------------------+-------+-----------------+-------------------+



In [13]:
usersdf_clean.show()

+-------+-------+---+------------------+
|user_id|   name|age|             email|
+-------+-------+---+------------------+
|      1| User_1| 45| user1@example.com|
|      2| User_2| 48| user2@example.com|
|      3| User_3| 36| user3@example.com|
|      4| User_4| 46| user4@example.com|
|      5| User_5| 29| user5@example.com|
|      6| User_6| 39| user6@example.com|
|      7| User_7| 24| user7@example.com|
|      8| User_8| 44| user8@example.com|
|      9| User_9| 27| user9@example.com|
|     10|User_10| 43|user10@example.com|
|     11|User_11| 36|user11@example.com|
|     12|User_12| 24|user12@example.com|
|     13|User_13| 47|user13@example.com|
|     14|User_14| 20|user14@example.com|
|     15|User_15| 30|user15@example.com|
|     16|User_16| 30|user16@example.com|
|     17|User_17| 35|user17@example.com|
|     18|User_18| 45|user18@example.com|
|     19|User_19| 47|user19@example.com|
|     20|User_20| 43|user20@example.com|
+-------+-------+---+------------------+
only showing top

In [14]:
purchasesdf_clean.show()

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
|          4|     72|        42|2022-01-04|       9|
|          5|     61|        44|2022-01-05|       6|
|          6|     21|        24|2022-01-06|       7|
|          7|     83|        15|2022-01-07|       7|
|          8|     87|        32|2022-01-08|       3|
|          9|     75|        32|2022-01-09|       2|
|         10|     75|        24|2022-01-10|       9|
|         11|     88|        41|2022-01-11|       8|
|         12|    100|        49|2022-01-12|       7|
|         13|     24|        49|2022-01-13|       9|
|         14|      3|        12|2022-01-14|       4|
|         15|     22|        39|2022-01-15|       4|
|         16|     53|         2|2022-01-16|   

In [15]:
productsdf_clean.show()

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
|         4|   Product_4|Electronics|  2.6|
|         5|   Product_5|Electronics|  9.4|
|         6|   Product_6|     Sports|  8.7|
|         7|   Product_7|     Beauty|  8.2|
|         8|   Product_8|     Sports|  1.0|
|         9|   Product_9|     Beauty|  6.0|
|        10|  Product_10|     Sports|  5.4|
|        11|  Product_11|     Sports|  8.6|
|        12|  Product_12|       Home|  1.3|
|        13|  Product_13|     Sports|  7.1|
|        14|  Product_14|   Clothing|  7.4|
|        16|  Product_16|     Sports|  4.3|
|        17|  Product_17|Electronics|  8.1|
|        18|  Product_18|   Clothing|  4.8|
|        19|  Product_19|   Clothing|  3.5|
|        20|  Product_20|   Clothing|  4.3|
|        21|  Product_21|       

In [16]:
prod_purch=purchasesdf_clean.join(productsdf_clean, productsdf_clean.product_id == purchasesdf_clean.product_id)

In [17]:
prod_purch_total=prod_purch.select(['category','price','quantity']).withColumn('total',col('price')*col('quantity')).groupby('category').sum('total')

In [18]:
prod_purch_total.show()

+-----------+------------------+
|   category|        sum(total)|
+-----------+------------------+
|       Home|1523.4999999999998|
|     Sports|1802.4999999999998|
|Electronics|1174.7999999999997|
|   Clothing|             790.3|
|     Beauty| 459.8999999999999|
+-----------+------------------+



In [19]:
prod_purch=prod_purch.withColumn("cost",col("price")*col("quantity"))
#select(['category','price','quantity']).groupby('category').sum(col('price')*col('quantity')).show()

In [20]:
prod_purch.select(['category','cost']).groupby('category').sum('cost').show()

+-----------+------------------+
|   category|         sum(cost)|
+-----------+------------------+
|       Home|1523.4999999999998|
|     Sports|1802.4999999999998|
|Electronics|1174.7999999999997|
|   Clothing|             790.3|
|     Beauty| 459.8999999999999|
+-----------+------------------+



In [21]:
users_prod_purch=usersdf_clean.join(prod_purch,usersdf_clean.user_id == prod_purch.user_id)

In [22]:
prod_purch_18_25=users_prod_purch.withColumn("cost",col("price")*col("quantity")).filter((col('Age')>18) & (col('Age')<25))

In [23]:
prod_purch_18_25.show()

+-------+-------+---+------------------+-----------+-------+----------+----------+--------+----------+------------+-----------+-----+------------------+
|user_id|   name|age|             email|purchase_id|user_id|product_id|      date|quantity|product_id|product_name|   category|price|              cost|
+-------+-------+---+------------------+-----------+-------+----------+----------+--------+----------+------------+-----------+-----+------------------+
|     58|User_58| 20|user58@example.com|         27|     58|        37|2022-01-27|       6|        37|  Product_37|   Clothing|  6.0|              36.0|
|     91|User_91| 21|user91@example.com|         31|     91|        19|2022-01-31|       6|        19|  Product_19|   Clothing|  3.5|              21.0|
|      7| User_7| 24| user7@example.com|         47|      7|        11|2022-02-16|       1|        11|  Product_11|     Sports|  8.6|               8.6|
|     73|User_73| 20|user73@example.com|         49|     73|        16|2022-02-18|

In [24]:
prod_purch_18_25_total=prod_purch_18_25.select(['category','cost']).groupby('category').sum('cost')

In [25]:
prod_purch_18_25_total.show()

+-----------+------------------+
|   category|         sum(cost)|
+-----------+------------------+
|       Home|174.70000000000002|
|     Sports|185.70000000000002|
|Electronics|143.60000000000002|
|   Clothing|             107.4|
|     Beauty|              33.2|
+-----------+------------------+



In [26]:
prod_purch_18_25_total=prod_purch_18_25_total.withColumnRenamed('sum(cost)','y_total')

In [27]:
prod_purch_total=prod_purch_total.withColumnRenamed('sum(total)','total').withColumnRenamed('category','category1')

In [28]:
differ_df=prod_purch_18_25_total.join(prod_purch_total,prod_purch_18_25_total.category==prod_purch_total.category1)

In [29]:
differ_df.show()

+-----------+------------------+-----------+------------------+
|   category|           y_total|  category1|             total|
+-----------+------------------+-----------+------------------+
|       Home|174.70000000000002|       Home|1523.4999999999998|
|     Sports|185.70000000000002|     Sports|1802.4999999999998|
|Electronics|143.60000000000002|Electronics|1174.7999999999997|
|   Clothing|             107.4|   Clothing|             790.3|
|     Beauty|              33.2|     Beauty| 459.8999999999999|
+-----------+------------------+-----------+------------------+



In [31]:
differ_df=differ_df.withColumn('part',col('y_total')/col('total')).drop(prod_purch_total.category1)

In [34]:
differ_df=differ_df.withColumn('part',round(differ_df.part,2))

In [35]:
differ_df.show()

+-----------+------------------+------------------+----+
|   category|           y_total|             total|part|
+-----------+------------------+------------------+----+
|       Home|174.70000000000002|1523.4999999999998|0.11|
|     Sports|185.70000000000002|1802.4999999999998| 0.1|
|Electronics|143.60000000000002|1174.7999999999997|0.12|
|   Clothing|             107.4|             790.3|0.14|
|     Beauty|              33.2| 459.8999999999999|0.07|
+-----------+------------------+------------------+----+



In [36]:
differ_df.orderBy(col('part').desc()).select('category','part').limit(3).show()

+-----------+----+
|   category|part|
+-----------+----+
|   Clothing|0.14|
|Electronics|0.12|
|       Home|0.11|
+-----------+----+

